# Using EcoFOCIpy to process raw field data

## NW2202 (Spring Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [5]:
sample_data_dir = '/Users/bell/ecoraid/2022/CTDcasts/nw2202/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [12]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'NW2202' #no hyphens
cruise_meta_file = sample_data_dir+'logs/NW2202.yaml'
inst_meta_file = sample_data_dir+'logs/EMA_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2022/CTDcasts/nw2202/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/nw2202/rawconverted/ctd002.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])


In [13]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,wetstar,t090c,t190c,prdm,cstarat0,cstartr0,scan,datetime
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4
mean,6.505575,98.300250,283.771250,30.837775,30.837600,23.848375,23.848125,6.452100,97.492250,281.439750,7.281700,9.081375,9.082075,10.861750,2.216150,64.673425,7450.750000,2022-09-01 10:02:23.750000128
min,6.500900,98.180000,283.564000,30.835200,30.835100,23.845000,23.844800,6.437000,97.280000,280.782000,6.290300,9.056800,9.058300,0.709000,1.300500,44.001200,6496.000000,2022-09-01 10:01:44
25%,6.502925,98.258750,283.655500,30.835275,30.835250,23.845150,23.845100,6.438050,97.298000,280.828500,6.599450,9.080725,9.081550,8.469250,1.306125,63.754550,7161.250000,2022-09-01 10:02:11.750000128
50%,6.505300,98.313000,283.761000,30.835600,30.835500,23.845500,23.845350,6.450250,97.481000,281.360000,7.221100,9.089050,9.089350,11.209500,1.357700,71.223750,7446.000000,2022-09-01 10:02:23.500000
75%,6.507950,98.354500,283.876750,30.838100,30.837850,23.848725,23.848375,6.464300,97.675250,281.971250,7.903350,9.089700,9.089875,13.602000,2.267725,72.142625,7735.500000,2022-09-01 10:02:35.500000
max,6.510800,98.395000,283.999000,30.844700,30.844300,23.857500,23.857000,6.470900,97.727000,282.257000,8.394300,9.090600,9.091300,20.319000,4.848700,72.245000,8415.000000,2022-09-01 10:03:04
std,0.004285,0.091889,0.188222,0.004627,0.004474,0.006093,0.005924,0.017021,0.233144,0.740667,0.960206,0.016402,0.015877,8.016793,1.755709,13.808798,785.134543,NaN


## Create BTL report file

In [14]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [15]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'NW2202',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NorthWest Explorer',
 'ShipID': 'NW',
 'StartDate': None,
 'EndDate': None,
 'Project': '',
 'ChiefScientist': '',
 'StartPort': '',
 'EndPort': '',
 'CruiseLocation': 'Bering',
 'Description': 'NBS',
 'CruiseYear': 2022,
 'ctdlogs_pdf_name': 'NW2202_CastLogs.pdf'}

In [16]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54004,
 'Vessel': 'NorthWest Explorer',
 'CruiseID': 'NW2202',
 'Project_Leg': '',
 'UniqueCruiseID': 'NW2202',
 'Project': 'Northern Bering Sea (NBS)',
 'StationNo_altname': '1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 60,
 'LatitudeMin': 0.3,
 'LongitudeDeg': 167,
 'LongitudeMin': 59.802,
 'GMTDay': 1,
 'GMTMonth': 'Sep',
 'GMTYear': 2022,
 'GMTTime': 65352,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': -9999,
 'SeaState': '',
 'Visibility': '',
 'WindDir': -999,
 'WindSpd': -99.9,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': -9999,
 'StationNameID': '',
 'MaxDepth': -9999,
 'InstrumentSerialNos': '',
 'Notes': 'Vacuum pumps had little to no pressufre throguhout filtering process. >5, >10, >20 filters still moist at completion of filtering',
 'NutrientBtlNiskinNo': '1;2;3;4;5;6;7;8;9;10',
 'NutrientBtlNumbers': ';;;;;;1;2;;3',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNis

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [17]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [18]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,wetstar,temperature_ch1,temperature_ch2,Pressure [dbar],Attenuation,Transmittance,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,
10.0,6.5009,98.18,283.564,30.8447,30.8443,23.8575,23.857,6.4709,97.727,282.257,8.3943,9.0568,9.0583,20.319,1.4074,70.339,6496.0,2022-09-01 10:01:44,ctd001


In [19]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'wetstar', 'temperature_ch1',
       'temperature_ch2', 'Pressure [dbar]', 'Attenuation', 'Transmittance',
       'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [20]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [21]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [22]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 4)
Coordinates:
  * bottle              (bottle) float64 10.0 9.0 8.0 7.0
Data variables: (12/19)
    oxy_concM_ch1       (bottle) float64 6.501 6.504 6.507 6.511
    oxy_percentsat_ch1  (bottle) float64 98.18 98.28 98.34 98.39
    oxy_conc_ch1        (bottle) float64 283.6 283.7 283.8 284.0
    salinity_ch1        (bottle) float64 30.84 30.84 30.84 30.84
    salinity_ch2        (bottle) float64 30.84 30.84 30.84 30.84
    sigma_t_ch1         (bottle) float64 23.86 23.85 23.84 23.85
    ...                  ...
    Pressure [dbar]     (bottle) float64 20.32 11.36 11.06 0.709
    Attenuation         (bottle) float64 1.407 1.308 1.3 4.849
    Transmittance       (bottle) float64 70.34 72.11 72.25 44.0
    scan                (bottle) float64 6.496e+03 7.383e+03 7.509e+03 8.415e+03
    datetime            (bottle) datetime64[ns] 2022-09-01T10:01:44 ... 2022-...
    cast                (bottle) object 'ctd001' 'ctd001' 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [23]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [24]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [25]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 4, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 10.0 9.0 8.0 7.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 6.501 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 98.18 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 283.6 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 30.84 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 30.84 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 23.86 ......
    ...                  ...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 20.32 ......
    Attenuation         (latitude, longitude, time, bottle) float64 1.407 ......
    Transmittance       (latitude, longitude, time, bottle) float64 70.34 ......
    scan                (latitude, longitude, time, bottle) float64 6.496e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....

In [26]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 4, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 10.0 9.0 8.0 7.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 6.501 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 98.18 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 283.6 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 30.84 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 30.84 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 23.86 ......
    ...                  ...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 20.32 ......
    Attenuation         (latitude, longitude, time, bottle) float64 1.407 ......
    Transmittance       (latitude, longitude, time, bottle) float64 70.34 ......
    scan                (latitude, longitude, time, bottle) float64 6.496e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....
Attributes: (12/44)
    CruiseID:                      NW2202
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NorthWest Explorer
    ShipID:                        NW
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-01-28T10:41:29Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [27]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history